In [4]:
import pandas as pd
import numpy as np
import random

In [3]:
!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip "/content/ml-latest-small.zip" -d "/content/movie_lens/"

--2022-04-28 07:40:06--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  --.-KB/s    in 0.1s    

2022-04-28 07:40:06 (8.27 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  /content/ml-latest-small.zip
   creating: /content/movie_lens/ml-latest-small/
  inflating: /content/movie_lens/ml-latest-small/links.csv  
  inflating: /content/movie_lens/ml-latest-small/tags.csv  
  inflating: /content/movie_lens/ml-latest-small/ratings.csv  
  inflating: /content/movie_lens/ml-latest-small/README.txt  
  inflating: /content/movie_lens/ml-latest-small/movies.csv  


# Load dataset

In [6]:
movie_data=pd.read_csv('/content/movie_lens/ml-latest-small/movies.csv')
rating_data=pd.read_csv('/content/movie_lens/ml-latest-small/ratings.csv')

In [10]:
movie_data

,movieId,title,genres,average_rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,NaN
1,2,Jumanji (1995),Adventure|Children|Fantasy,3.920930
2,3,Grumpier Old Men (1995),Comedy|Romance,3.431818
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,3.259615
4,5,Father of the Bride Part II (1995),Comedy,2.357143
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,NaN
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,NaN
9739,193585,Flint (2017),Drama,NaN
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,NaN


In [11]:
len(movie_data)

9742

# Content based recommendation system

In [14]:
movie_data['average_rating']=rating_data.groupby('movieId').mean().rating     # adds a new column to movie_data which has average ratings of movies
new_dict={}       # new_dict will contain genre,list len(list)=2 1st pos has sum of all ratings of that genre 2nd pos counts the no of times added
for i in range(len(movie_data)):      # computes the rating of each movie and adds it to all the genres of the movie also counts 
     for genre in movie_data.iloc[i,2].split('|'):
            if np.isnan(movie_data.iloc[i,3]):
                break
            if genre in new_dict.keys():        # computes the rating of each movie and adds it to all the genres
                new_dict[genre][0]+=movie_data.iloc[i,3]
                new_dict[genre][1]+=1
            else:
                new_dict[genre] = [0,0]             # counts the no of times added to each genre will be used for average
                new_dict[genre][0]=movie_data.iloc[i,3]
                new_dict[genre][1]=1
for tag in new_dict:                    #changes new dict to genre,avg rating
    new_dict[tag]=(new_dict[tag][0]/new_dict[tag][1])
    
genreavg=pd.DataFrame.from_dict(new_dict,orient='index',columns=['rating']).sort_values('rating',ascending=False) #  creating a new dataframe of new_dict so that we can sort them by average rating as dictionary cant be ordered
   
rand_ind=random.choices(range(10329),k=3)   # 3 three random movies for user to rate

dicti={}            # contains movies and thier ratings 

print("please rate the below movies on the scale 1-5 and rate as 0 if not seen")  
for index in rand_ind:  # if user has not seen then another movie must be given but the new movie must not alredy be given for the user to rate
    print(movie_data.iloc[index,1])
    flag = 1
    while True:  # parent while
        m=int(input())
        if m!=0:
            if flag == 1:
                dicti[movie_data.iloc[index,1]]=m
            else:
                dicti[movie_data.iloc[randint,1]]=m  # randint will be defined only if he gives 0 to any movie
            break
        else:
            while True:  # child while
                randint=random.choice(range(10329))
                if randint not in rand_ind:         #  makes sure the new movie is not in the first three
                    if (movie_data.iloc[randint,1] not in dicti.keys()) or dicti.keys()==[]: # rated movie should be given again to rate
                        print(movie_data.iloc[randint,1])  
                        flag = 0                    # it will go directly into the else of parent while
                        break

temp_dict={}  # (genre,[rating,count]) to contain the average rating of all genres present in the 3 movies which he rated 
sum=0
i=0

for movie,rating in dicti.items(): # iterates to each movie and adds the rating considering it as rating of each individual genre
    for agenre in movie_data[movie_data.title==movie].genres.values[0].split("|"): 
        if agenre in temp_dict.keys():
            temp_dict[agenre][0]+=rating
            temp_dict[agenre][1]+=1
        else:
            temp_dict[agenre]=[0,0]
            temp_dict[agenre][0]=rating
            temp_dict[agenre][1]=1

temp_dict1={} # to have genres whose rating is higher than the average rating of that genre considering the whole dataset

for genre,some_list in temp_dict.items(): 
    temp_dict[genre]=(some_list[0]/some_list[1])
    
for genre in temp_dict.keys():  # removes genres which have alredy been in the 3 movies
    genreavg.drop(genre,inplace=True)
    
for genre in temp_dict:    # removes genres with less ratings than the whole data
    if new_dict[genre]<temp_dict[genre]:
        temp_dict1[genre]=temp_dict[genre]
        
temp_dict=temp_dict1

'''weights rating into fractionals '''
for cumrating in temp_dict.values():
    sum+=cumrating
for movie,cumrating in temp_dict.items():
    temp_dict[movie] = cumrating/sum
'''                                      '''

temp_list=[]       # will contain the final 20 recommended movies based on genre and inside each genre movies are in desecnding order by average rating
print()
print("Movie genre stats for recommendation")
for genre,times in temp_dict.items():
    nr = 5  # no of random movies to recommned
    n=round(int((20-nr)*(times)))          # n= no of movies to be taken from that genre  !!! CHANGE THIS 15 TO 20 IF ALL MOVIES SHOULD BE GIVEN BY USERS CHOICE
    print(n,"movies must be taken from",genre)
    for i in range(n):
        temp_list.append(movie_data[(movie_data['genres'].str.count(genre)>=1)].sort_values(by=['average_rating'],ascending=False).iloc[0:n].title.iloc[i])  # takes the first top n movies of highest rating in that genre
        
for genre in genreavg.index[0:5]:  # takes top 5 genres based on overall average rating 
    data=movie_data[(movie_data['genres'].str.count(genre)>=1)].sort_values(by=['average_rating'],ascending=False)  # the movies of a genre ordered on average rating in descending
    max_rating=data.iloc[0,3]   # max rating will be the average rating of the first movie in data
    randId=random.choices(data[data.average_rating==max_rating]['movieId'].values)  # gives a random movieId of all movies with max rating of that genre
    temp_list.append(movie_data[movie_data['movieId']==randId[0]]['title'].values[0]) # added to our final list

while True:  # Final check to remove any repeted movies present in temp_list and replace them with random movies or to fill if no of movies in temp_list is less than 20
    
    randmov=random.choices(movie_data.title,k=20-len(temp_list))
    temp_list=temp_list+randmov
    
    for movie in dicti.keys():
        if movie in temp_list:
            temp_list.remove(movie)  # removes movies if present in the rated movies by the user
            
    l=temp_list[::-1] # list inverted cause for repeted movies we need to remove the movie which is lesser in list order but python remove() removes first encoutered which may result in higher prefernce movie going down in order which is not desirable
    
    for movie in temp_list:   # removes repeted movies by counting no of times movie is repeted
        n=l.count(movie)
        for i in range(n-1):
            l.remove(movie)
    temp_list=l[::-1]
    
    if len(temp_list)==20:    # till 20 unique movies is stored it will keep on adding random movies and removing repeted ones
        break
        
print()     
print("Recommended movies:") 
for movie in temp_list:
    print(movie)

please rate the below movies on the scale 1-5 and rate as 0 if not seen
Me Without You (2001)
3
Two Night Stand (2014)
4
9/11 (2002)
1

Movie genre stats for recommendation
7 movies must be taken from Comedy
8 movies must be taken from Romance

Recommended movies:
Company Man (2000)
Band of Outsiders (Bande à part) (1964)
Divine Secrets of the Ya-Ya Sisterhood (2002)
Johnny Dangerously (1984)
Family Plot (1976)
Rumble in the Bronx (Hont faan kui) (1995)
La vérité si je mens ! (1997)
Mr. Deeds Goes to Town (1936)
Joe Versus the Volcano (1990)
Between Your Legs (Entre las piernas) (1999)
Clash of the Titans (1981)
Hot Shots! (1991)
Chinese Ghost Story, A (Sinnui yauwan) (1987)
Hoosiers (a.k.a. Best Shot) (1986)
Norbit (2007)
Green Lantern: First Flight (2009)
Philadelphia Experiment, The (1984)
San Francisco (1936)
Big Red One, The (1980)
Angels with Dirty Faces (1938)
